In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

## RAG 절차
- 준비 : https://law.go.kr/법령/소득세법 에서 doc다운로드(hwp는 파이썬 못 읽음. pdf는 한글의 경우 단어가 짤림) 받아 파일형식을 docx로 변경

1. 문서를 읽는다(python-docx이용) : \n은 제외
2. 읽어온 문서를 쪼갠다(tiktoken 이용)
    * 모델의 context window 확인
    * chunk(문서를 쪼갠 하나)가 길면 비용과 시간이 많이 들어
3. 쪼갠 문서를 임베딩 -> vector database에 저장 -> chroma(local vector database==vector store)/pinecorn(클라우드 vector database=vector store)
4. 질문과 vector DB의 유사도 검색                                              
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달하여 답변 생성


# 1. 문서를 읽는다(python-docx이용)
- pip install python-docx
- pip install langchain-community

In [2]:
%pip install -U -q langchain-community docx2txt python-docx

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 이렇게 docx파일을 읽어오면 하나로 한꺼번에 가져옴(\n포함)
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
document = loader.load() #한꺼번에 load
#document

In [6]:
from docx import Document
document = Document('./data/소득세법(법률)(제21065호)(20260102).docx')
print(document)
print(dir(document)) # 객체가 가지고 있는 속성이름

['_Document__body', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_block_width', '_body', '_element', '_parent', '_part', 'add_comment', 'add_heading', 'add_page_break', 'add_paragraph', 'add_picture', 'add_section', 'add_table', 'comments', 'core_properties', 'element', 'inline_shapes', 'iter_inner_content', 'paragraphs', 'part', 'save', 'sections', 'settings', 'styles', 'tables']


In [10]:
print(len(document.paragraphs)) # \n마다 새로운 paragraph로 생성
for paragraph in document.paragraphs[:5]:
    print(dir(paragraph))

3182
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_element', '_insert_paragraph_before', '_p', '_parent', 'add_run', 'alignment', 'clear', 'contains_page_break', 'hyperlinks', 'insert_paragraph_before', 'iter_inner_content', 'paragraph_format', 'part', 'rendered_page_breaks', 'runs', 'style', 'text']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_element', '_insert_paragraph_before', '_p', '_pare